# Default notebook

This default notebook is executed using Databricks Workflows as defined in resources/delta_live_store_job.yml.

In [ ]:
%pip install -qr ../requirements-dev.txt 
%pip install -q --upgrade databricks-sdk
dbutils.library.restartPython()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dbrx.dls.store import DeltaLiveStore, DeltaLiveEntity
from dbrx.dls.filters import by_entity_id, Predicate

In [ ]:
dbutils.widgets.text("sourcePath", "")
dbutils.widgets.text("destinationTable", "")
dbutils.widgets.text("deltaLiveStoreTable", "")

In [ ]:
source_path: str = dbutils.widgets.get("sourcePath")
destination_table: str = dbutils.widgets.get("destinationTable")
delta_live_store_table: str = dbutils.widgets.get("deltaLiveStoreTable")

print(f"source_path: {source_path}")
print(f"destination_table: {destination_table}")
print(f"delta_live_store_table: {delta_live_store_table}")

if source_path is None:
    raise ValueError("source_path is not set")
if destination_table is None:
    raise ValueError("destination_table is not set")
if delta_live_store_table is None:
    raise ValueError("delta_live_store_table is not set")


In [ ]:

def new_delta_live_entity() -> DeltaLiveEntity:
    return DeltaLiveEntity(
        entity_id="1",
        source=source_path,
        destination=destination_table,
        source_format="cloudFiles",
        read_options={
            "cloudFiles.format": "csv",
            "header": "true"
        },
    )

In [ ]:

store: DeltaLiveStore = DeltaLiveStore(delta_live_store_table)

In [ ]:
store.is_initialized()

In [ ]:
store.initialize()

In [ ]:
store.upsert(new_delta_live_entity())

predicate: Predicate = by_entity_id("1") 
display(store.find_entity(predicate))